# **Sistemas Multi-Agente y ChromaDB con LlamaIndex** 🤖🗄️

Este notebook forma parte del curso de **IA Generativa** de la [Fundación GoodJob](https://www.fundaciongoodjob.org/).  
En él, los alumnos aprenderán a crear **sistemas multi-agente** avanzados que pueden trabajar con **bases de datos vectoriales** y **múltiples fuentes de información**.  

<p align="center">
  <img src="../notebooks/sources/llamaindex.png" alt="LlamaIndex" style="width:100%; height:auto;"/>
</p>


## ¿Qué vamos a aprender hoy? 🎯

En el notebook anterior aprendimos a crear agentes básicos con herramientas simples. ¡Ahora vamos a dar un salto a agentes más complejos! 

Hoy construiremos:

- 🗄️ **Bases de datos vectoriales** con ChromaDB para almacenar conocimiento
- 👥 **Sistemas multi-agente** con especialistas para diferentes tareas
- 🧠 **Agentes inteligentes** que pueden consultar documentos, APIs y hacer cálculos
- 🎯 **Routers** que deciden qué agente usar para cada consulta
- 🔄 **Orquestación** para combinar múltiples fuentes de información

¡Empezamos esta aventura! 🌟


## Prerrequisitos y configuración inicial 🔧

Antes de empezar, necesitamos instalar algunas dependencias adicionales para trabajar con bases de datos vectoriales y sistemas multi-agente.


In [3]:
# Install additional dependencies if needed
# Check if we have everything we need
import os
import sys
from pathlib import Path

print("🔍 Verificando configuración del entorno...")

# Check if data directory exists
data_dir = Path("../data")
if not data_dir.exists():
    print("📁 Creando directorio de datos...")
    data_dir.mkdir(exist_ok=True)
    (data_dir / "pdfs").mkdir(exist_ok=True)
    print("✅ Directorios creados: ../data/pdfs")
else:
    print("✅ Directorio de datos encontrado")

# Check if ChromaDB directory exists
chroma_dir = Path("../chroma_db")
if not chroma_dir.exists():
    print("🗄️ Preparando directorio para ChromaDB...")
    chroma_dir.mkdir(exist_ok=True)
    print("✅ Directorio ChromaDB creado")
else:
    print("✅ Directorio ChromaDB encontrado")

print("🎉 ¡Configuración inicial completada!")


🔍 Verificando configuración del entorno...
✅ Directorio de datos encontrado
✅ Directorio ChromaDB encontrado
🎉 ¡Configuración inicial completada!


## 1. ¿Qué es una base de datos vectorial? 🧠

### El problema que resuelven

Imagina que tienes **miles de documentos PDF** con información importante de tu empresa. Un usuario te pregunta: *"¿Cuál es la política de reembolsos para clientes internacionales?"*

Con métodos tradicionales tendrías que:
1. 😰 Buscar manualmente en cada documento
2. 🔍 Usar búsqueda por palabras clave (que puede fallar si usan sinónimos)
3. ⏰ Invertir horas en encontrar la respuesta

### La solución: Bases de datos vectoriales

Una **base de datos vectorial** convierte tus textos en **vectores numéricos** (embeddings) que capturan el **significado semántico**. Esto permite:

- 🎯 **Búsqueda semántica**: Encuentra contenido por significado, no solo palabras exactas
- ⚡ **Velocidad**: Consultas en milisegundos, incluso con millones de documentos  
- 🧠 **Inteligencia**: Entiende sinónimos, contexto y relaciones conceptuales
- 💾 **Persistencia**: Guarda todo para reutilizar sin reprocesar

### ¿Por qué ChromaDB?

**ChromaDB** es perfecto para aprender y proyectos porque es:
- 🚀 **Fácil de instalar**: Un simple `pip install`
- 💾 **Persistente**: Guarda tus datos localmente
- 🔧 **Integrado**: Funciona perfectamente con LlamaIndex
- 🆓 **Gratuito**: Sin costos de infraestructura

```
📄 PDF → 📝 Chunks → 🧮 Embeddings → 🗄️ ChromaDB → 🔍 Consulta → 🤖 LLM → ✨ Respuesta
```

Aunque en este curso utilizaremos **ChromaDB** por su simplicidad y porque requiere pocos recursos, existen muchas otras opciones muy válidas y potentes para bases de datos vectoriales. Algunas alternativas populares incluyen:
 
 - [Qdrant](https://qdrant.tech/): Motor de base de datos vectorial de alto rendimiento, open source y con muchas integraciones.
 - [Weaviate](https://weaviate.io/): Plataforma de base de datos vectorial con capacidades de búsqueda semántica y módulos de IA integrados.
 - [Pinecone](https://www.pinecone.io/): Servicio gestionado en la nube, muy utilizado en producción.
 - [Milvus](https://milvus.io/): Sistema open source escalable para gestión de vectores.
 - [PostgreSQL + pgvector](https://github.com/pgvector/pgvector): Extensión para PostgreSQL que permite almacenar y consultar vectores en una base de datos relacional tradicional.
 
Todas estas opciones son ampliamente utilizadas en la industria y pueden ser más adecuadas para proyectos en producción o con necesidades específicas.



## 1.1 ¿Qué es Top-K en Búsqueda Vectorial? 🎯

El **Top-K** es un concepto fundamental en las bases de datos vectoriales que determina **cuántos documentos similares** devolver para cada consulta.

### 🔍 ¿Cómo funciona Top-K?

Cuando haces una consulta en una base de datos vectorial:

1. **📝 Tu consulta** se convierte en un vector (embedding)
2. **📊 Se calcula la similitud** entre tu vector y todos los vectores almacenados
3. **🏆 Se ordenan por similitud** (más similar = menor distancia)
4. **🎯 Se devuelven los K más similares** (donde K es el número que configures)

<p align="center">
  <img src="sources/tok_k.jpg" alt="Funcionamiento Top-K en Base de Datos Vectorial" style="width:80%; height:auto;"/>
</p>

**Flujo de datos:**
- Los **Text Chunks** (documentos fragmentados) se procesan con **Encoders**
- Cada chunk genera su **Embedding** correspondiente
- Todos los embeddings se almacenan en la **Vector DB**
- Al consultar, se comparan similitudes y se devuelven los **Top-K más relevantes**

### ⚙️ Configurando Top-K: ¿Qué valor usar?

| **Top-K** | **Uso Recomendado** | **Ventajas** | **Desventajas** |
|-----------|-------------------|-------------|----------------|
| **1-3** | Respuestas muy específicas | 🎯 Máxima precisión, respuestas concisas | ⚠️ Puede perder contexto importante |
| **3-5** | Uso general (Recomendado) | ⚖️ Buen balance precisión/contexto | ✅ Ideal para la mayoría de casos |
| **5-8** | Análisis profundo | 📚 Contexto rico, análisis completo | 🐌 Respuestas más largas |
| **8+** | Investigación exhaustiva | 🔍 Contexto máximo | ⚠️ Riesgo de información irrelevante |

### 💡 Factores que Influyen en Top-K

1. **📏 Longitud del contexto del LLM**: Modelos con más contexto pueden procesar más chunks
2. **🎯 Tipo de consulta**: Preguntas específicas necesitan menos chunks
3. **📊 Calidad de los embeddings**: Mejores embeddings = mejor precisión con menos chunks
4. **⚡ Rendimiento**: Más chunks = más tiempo de procesamiento



## 1.2 ¿Cómo se Calcula la Similitud entre Embeddings? 📐

Para que una base de datos vectorial funcione, necesita una forma de **medir qué tan similares** son dos vectores. La métrica más común y efectiva es la **similitud del coseno**.

### 🧮 Similitud del Coseno: La Matemática Detrás de la Magia

La similitud del coseno mide el **ángulo entre dos vectores**, no su magnitud. Esto es perfecto para embeddings porque nos interesa la **dirección** (significado) más que la "intensidad".

<p align="center">
  <img src="sources/cosine_similarity.png" alt="Cálculo de Similitud del Coseno" style="width:85%; height:auto;"/>
</p>


#### Fórmula Matemática:

$
\cos(\theta) = \frac{A \cdot B}{\|A\| \, \|B\|}
$


Donde:
- **A · B**: Producto de los vectores
- **||A||**: Magnitud (norma) del vector A  
- **||B||**: Magnitud (norma) del vector B
- **θ**: Ángulo entre los vectores

### 🎯 ¿Por qué Coseno y no Distancia Euclidiana?

#### ✅ Similitud del Coseno (Recomendada)
- 🎯 **Insensible a magnitud**: Solo importa la dirección
- 📝 **Ideal para texto**: Captura significado semántico
- 🔢 **Rango normalizado**: Siempre entre 0 y 1
- ⚡ **Eficiente**: Cálculo optimizado

#### ❌ Distancia Euclidiana (Menos ideal para texto)
- 📏 **Sensible a magnitud**: Vectores largos parecen más diferentes
- 🔍 **Mejor para coordenadas**: Ideal para datos espaciales
- 📊 **Rango variable**: No está normalizada



## 1.3 Tipos de conexiones de bases de datos vectoriales con ChromaDB

ChromaDB ofrece múltiples formas de conectarse y almacenar datos, cada una optimizada para diferentes casos de uso. Vamos a explorar las tres opciones principales:

### 🔧 Opción 1: ChromaDB Persistente (Recomendado para desarrollo)

Esta es la opción que usaremos principalmente en el curso. Los datos se guardan en disco y persisten entre sesiones:

**Ventajas:**
- 💾 **Persistencia**: Los datos se mantienen entre sesiones
- 🔄 **Reutilización**: No necesitas reprocesar documentos
- 🏠 **Local**: Todo funciona sin conexión a internet
- 📁 **Organización**: Fácil backup y gestión de datos

**Cuándo usar:**
- Desarrollo y experimentación
- Proyectos personales
- Cuando necesitas persistir datos localmente

In [5]:
# OPCIÓN 1: ChromaDB Persistente - Los datos se guardan en disco
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

# Crear cliente persistente (los datos se guardan en ../chroma_db)
client = chromadb.PersistentClient(path="../chroma_db")
collection = client.get_or_create_collection("company_policies_persistent")

print("✅ ChromaDB Persistente configurado")
print("💾 Los datos se guardarán en: ../chroma_db")
print("🔄 Los datos persistirán entre sesiones")


✅ ChromaDB Persistente configurado
💾 Los datos se guardarán en: ../chroma_db
🔄 Los datos persistirán entre sesiones


### ⚡ Opción 2: ChromaDB en Memoria (Para experimentos rápidos)

Ideal para testing y experimentos donde no necesitas persistir los datos:

**Ventajas:**
- ⚡ **Velocidad máxima**: Todo en RAM
- 🧪 **Testing**: Perfecto para pruebas rápidas
- 🧹 **Limpieza automática**: No deja archivos residuales
- 🔄 **Reinicio limpio**: Cada ejecución empieza desde cero

**Cuándo usar:**
- Pruebas y experimentos temporales
- Desarrollo de prototipos
- Cuando no necesitas persistir datos
- Testing automatizado


In [6]:
# OPCIÓN 2: ChromaDB en Memoria - Máxima velocidad, datos temporales
import chromadb

# Crear cliente en memoria (los datos se pierden al reiniciar)
client_memory = chromadb.Client()  # Sin path = en memoria
collection_memory = client_memory.get_or_create_collection("temp_policies")

print("✅ ChromaDB en Memoria configurado")
print("⚡ Máxima velocidad de acceso")
print("⚠️ Los datos se perderán al reiniciar el programa")

# Ejemplo de uso rápido
collection_memory.add(
    documents=["Documento de prueba en memoria"],
    ids=["test_1"]
)

# Se usará un modelo de embeddings local para ahorrar costes
results = collection_memory.query(
    query_texts=["documento"],
    n_results=1
)
print(f"🧪 Test exitoso: {len(results['documents'][0])} documento encontrado")


✅ ChromaDB en Memoria configurado
⚡ Máxima velocidad de acceso
⚠️ Los datos se perderán al reiniciar el programa
🧪 Test exitoso: 1 documento encontrado


### 🐳 Opción 3: ChromaDB con Docker (Para producción)

Docker permite ejecutar ChromaDB como un servicio independiente, ideal para producción y equipos:

**¿Por qué Docker?**
- 🔒 **Aislamiento**: La base de datos corre en su propio contenedor
- 🚀 **Escalabilidad**: Fácil de escalar y gestionar recursos
- 👥 **Colaboración**: Mismo entorno para todo el equipo
- 🔄 **CI/CD**: Integración perfecta con pipelines de despliegue
- 🌐 **Acceso remoto**: Múltiples clientes pueden conectarse

**Comandos Docker para ChromaDB:**

```bash
# 1. Ejecutar ChromaDB en Docker
docker run -p 8000:8000 chromadb/chroma

# 2. Con persistencia de datos
docker run -p 8000:8000 -v ./chroma_data:/chroma/chroma chromadb/chroma

# 3. Con docker-compose (recomendado para producción)
# Crear archivo docker-compose.yml con configuración completa
```

**Conexión Python**


In [7]:
# OPCIÓN 3: ChromaDB con Docker - Para producción y equipos
import chromadb

# Conectar a ChromaDB ejecutándose en Docker
# Nota: Primero debes ejecutar: docker run -p 8000:8000 chromadb/chroma

try:
    # Conectar al servidor ChromaDB en Docker
    client_docker = chromadb.HttpClient(host='localhost', port=8000)
    collection_docker = client_docker.get_or_create_collection("production_policies")
    
    print("✅ ChromaDB Docker configurado")
    print("🐳 Conectado al contenedor en localhost:8000")
    print("🏢 Listo para producción")
    
except Exception as e:
    print("⚠️ ChromaDB Docker no disponible")
    print("💡 Para usar esta opción, ejecuta: docker run -p 8000:8000 chromadb/chroma")
    print(f"📝 Error: {e}")

# Ejemplo de docker-compose.yml para producción:
docker_compose_example = """
version: '3.8'
services:
  chromadb:
    image: chromadb/chroma:latest
    ports:
      - "8000:8000"
    volumes:
      - ./chroma_data:/chroma/chroma
    environment:
      - CHROMA_SERVER_HOST=0.0.0.0
      - CHROMA_SERVER_PORT=8000
"""

print("📋 Ejemplo de docker-compose.yml:")
print(docker_compose_example)


⚠️ ChromaDB Docker no disponible
💡 Para usar esta opción, ejecuta: docker run -p 8000:8000 chromadb/chroma
📝 Error: Could not connect to a Chroma server. Are you sure it is running?
📋 Ejemplo de docker-compose.yml:

version: '3.8'
services:
  chromadb:
    image: chromadb/chroma:latest
    ports:
      - "8000:8000"
    volumes:
      - ./chroma_data:/chroma/chroma
    environment:
      - CHROMA_SERVER_HOST=0.0.0.0
      - CHROMA_SERVER_PORT=8000



> ### 💡 Consejos Prácticos  
> Para desarrollo local usa Persistente (cambia a Memoria en tests rápidos), colabora con Docker para compartir configuración, recuerda que solo Persistente y Docker mantienen datos entre reinicios, y ten en cuenta que en rendimiento: Memoria > Docker > Persistente.  




## 2. Configurando nuestro LLM y embeddings 🔧

Primero, vamos a configurar nuestro modelo de lenguaje y el modelo de embeddings que convertirá nuestros textos en vectores.


In [ ]:
from huggingface_hub import InferenceClient

from sklearn.metrics.pairwise import cosine_similarity


def get_embedding(text: str) -> list[float]:
    client = InferenceClient(
    provider="nebius"
    )

    result = client.feature_extraction(
        text,
        model="Qwen/Qwen3-Embedding-8B",
    )
    return result[0]

king_embedding = get_embedding("King")
queen_embedding = get_embedding("Queen")
cat_embedding = get_embedding("Cat")

print(cosine_similarity([king_embedding], [queen_embedding]))
print(cosine_similarity([king_embedding], [cat_embedding]))



[[0.7313868]]
[[0.5848896]]


> ⚠️ Nota para el curso: Este bloque solo debe ejecutarse si tienes créditos en Hugging Face o si deseas usar un modelo local.


In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.embeddings.base_embedding import HuggingFaceInferenceEmbedding
from dotenv import load_dotenv


# Configure embeddings - using a LOCAL open model . Requiere GPU!!
model_name = "Qwen/Qwen3-Embedding-8B"
# Settings.embed_model = HuggingFaceEmbedding(
#     model_name= model_name
# )
Settings.embed_model = HuggingFaceInferenceEmbedding(
    model=model_name,
    provider="nebius"
)

model_name = "text-embedding-ada-002"
Settings.embed_model = OpenAIEmbedding(embed_batch_size=10)

print(f"✅ Modelo de embeddings configurado: {model_name}")
print("🎉 ¡Configuración completada!")

✅ Modelo de embeddings configurado: text-embedding-3-small
🎉 ¡Configuración completada!


<div align="center">
  <iframe src="https://projector.tensorflow.org/" width="1000" height="600"></iframe>
</div>


## 3. Creando nuestra primera base de datos vectorial 🗄️

Ahora vamos a crear una base de datos vectorial con ChromaDB. Para este ejemplo, vamos a crear algunos documentos de prueba que simulen políticas empresariales.
Más adelante en el curso profundizaremos en el parseo seguro de documentos utilizando `Pydantic` para garantizar la validez y estructura de los datos.


In [12]:
from llama_index.core import Document, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Create sample documents (in a real scenario, you'd load PDFs)
sample_docs = [
    Document(text="""
    POLÍTICA DE TRANSFERENCIAS INTERNACIONALES
    
    Las transferencias SWIFT a países de la Unión Europea tienen una comisión fija de 15 EUR 
    más el 0.2% del importe transferido. Para transferencias a Estados Unidos, la comisión 
    es de 25 USD más el 0.3% del importe. Las transferencias SEPA dentro de la eurozona 
    son gratuitas para importes superiores a 1000 EUR, y tienen una comisión de 2 EUR 
    para importes inferiores.
    """),
    
    Document(text="""
    POLÍTICA DE REEMBOLSOS Y DEVOLUCIONES
    
    Los reembolsos se procesan en un plazo máximo de 5 días laborables para clientes 
    nacionales y 10 días laborables para clientes internacionales. Se aplica una 
    comisión de gestión del 1% sobre el importe reembolsado, con un mínimo de 5 EUR. 
    Los reembolsos superiores a 10,000 EUR requieren aprobación del departamento de 
    riesgo y pueden tardar hasta 15 días laborables.
    """),
    
    Document(text="""
    TIPOS DE CAMBIO Y CONVERSIONES
    
    Los tipos de cambio se actualizan cada hora durante el horario comercial. 
    Para conversiones EUR/USD se aplica un spread del 0.5%, para EUR/GBP del 0.3% 
    y para otras divisas del 0.8%. Las conversiones automáticas se realizan al 
    tipo de cambio vigente en el momento de la transacción más el spread correspondiente.
    """)
]

print(f"📄 Documentos de ejemplo creados: {len(sample_docs)}")

# Parse documents into chunks
parser = SentenceSplitter(chunk_size=500, chunk_overlap=50)
nodes = parser.get_nodes_from_documents(sample_docs)
print(f"📝 Chunks generados: {len(nodes)}")

# Create ChromaDB client and collection
client = chromadb.PersistentClient(path="../chroma_db")
collection = client.get_or_create_collection("company_policies")

# Create vector store and index
vector_store = ChromaVectorStore(chroma_collection=collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index (this will generate embeddings and store them)
print("🧮 Generando embeddings y almacenando en ChromaDB...")
index_docs = VectorStoreIndex(nodes, storage_context=storage_context)

# Create query engine
qe_docs = index_docs.as_query_engine(similarity_top_k=3)

print("✅ ¡Base de datos vectorial creada y lista para consultas!")
print("💾 Los datos están persistidos en ../chroma_db")


📄 Documentos de ejemplo creados: 3
📝 Chunks generados: 3
🧮 Generando embeddings y almacenando en ChromaDB...
✅ ¡Base de datos vectorial creada y lista para consultas!
💾 Los datos están persistidos en ../chroma_db


### ¡Probemos nuestra base de datos vectorial! 🔍

Ahora vamos a hacer algunas consultas para ver cómo funciona la búsqueda semántica:


In [5]:
# Test our vector database with semantic search
print("🔍 Probando búsqueda semántica en nuestra base de datos vectorial...\n")

# Test queries
test_queries = [
    "¿Cuánto cuesta enviar dinero a Estados Unidos?",
    "¿Cuál es el tiempo de procesamiento para devoluciones internacionales?",
    "¿Qué spread se aplica para cambiar euros a libras?"
]

for i, query in enumerate(test_queries, 1):
    print(f"📝 Consulta {i}: {query}")
    response = qe_docs.query(query)
    print(f"🤖 Respuesta: {response.response}")
    print("-" * 80)

print("✨ ¡Increíble! La base de datos vectorial entiende el significado de nuestras preguntas")


🔍 Probando búsqueda semántica en nuestra base de datos vectorial...

📝 Consulta 1: ¿Cuánto cuesta enviar dinero a Estados Unidos?
🤖 Respuesta: Sending money to the United States costs 25 USD plus 0.3% of the transfer amount.
--------------------------------------------------------------------------------
📝 Consulta 2: ¿Cuál es el tiempo de procesamiento para devoluciones internacionales?
🤖 Respuesta: El tiempo de procesamiento para devoluciones internacionales es de 10 días laborables.
--------------------------------------------------------------------------------
📝 Consulta 3: ¿Qué spread se aplica para cambiar euros a libras?
🤖 Respuesta: El spread del 0.3% se aplica para cambiar euros a libras.
--------------------------------------------------------------------------------
✨ ¡Increíble! La base de datos vectorial entiende el significado de nuestras preguntas



Según la documentación oficial, el `ResponseSynthesizer` es el componente que genera la respuesta final usando un LLM, a partir de los fragmentos recuperados y la consulta del usuario.

Si no defines un LLM explícitamente ni un response_synthesizer, LlamaIndex utiliza los valores globales por defecto de Settings:
   - El LLM por defecto es Settings.llm (si no lo configuras, llama-index impondrá un modelo básico).
   - El modo de síntesis de respuesta por defecto es 'compact', aunque hay otros modos distintos que puedes explorar (ver en la [documentación](https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/)).
 


## 4 Sistemas Multi-agente 🤖
Los sistemas multiagente en LlamaIndex permiten coordinar múltiples agentes inteligentes, cada uno especializado en tareas o dominios específicos, para resolver problemas complejos de manera colaborativa. Estos sistemas combinan modelos de lenguaje, herramientas y flujos de trabajo personalizados, facilitando la creación de soluciones avanzadas y adaptables en entornos empresariales y de investigación.

### 4.1 Fundamentos de Agentes en LlamaIndex 

Antes de crear nuestro sistema multi-agente, es importante entender la **anatomía de agentes** en LlamaIndex.

#### 🧠 Anatomía de un Agente

Todos los agentes en LlamaIndex tienen estos componentes clave:

```
🤖 AGENTE = 🧠 LLM + 🛠️ Tools + 📋 System Prompt + 🔄 Workflow Logic
```

- **🧠 LLM**: El "cerebro" que toma decisiones
- **🛠️ Tools**: Las "manos" que ejecutan acciones  
- **📋 System Prompt**: Las "instrucciones" de comportamiento
- **🔄 Workflow Logic**: El "proceso" de razonamiento

##### 🎯 Agentes Especializados vs Generales

**🎯 Agentes Especializados** (Recomendado):
- Un agente por dominio específico
- Mayor precisión en su área
- Más fácil de debuggear y optimizar

**🌐 Agentes Generales**:
- Un agente con todas las herramientas
- Más simple de configurar
- Puede confundirse con muchas opciones y/o contextos largos


### 4.2 Creando nuestros agentes especialistas 👥

Ahora viene la parte emocionante: vamos a crear un equipo de agentes especialistas. Cada uno será experto en un área específica:
- 💸 TransferenciasAgent: Especialista en transferencias bancarias
- 💰 ReembolsosAgent: Experto en gestión de reembolsos  
- 💱 CambiosAgent: Especialista en tipos de cambio y conversiones

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata, FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow
from llama_index.core.workflow import Context

from typing import Literal, Optional
from llama_index.core import Settings

from llama_index.llms.openai import OpenAI


llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    # api_key="some key",  # uses OPENAI_API_KEY env var by default
)
Settings.llm = llm

# 1. DOCS AGENT TOOLS - Wrapping our query engine as a tool
policy_tool = QueryEngineTool.from_defaults(
    query_engine=qe_docs,
    name="policies_search",
    description=(
        "Busca información en las políticas y documentos de la empresa sobre "
        "transferencias, reembolsos, tipos de cambio y comisiones."
    ),
)

# ---------------------------------------------------------------------------
# Tools de CÁLCULO (para complementar al RAG)
# ---------------------------------------------------------------------------
def calcular_comision_swift(region: Literal["UE","US"], importe: float) -> str:
    """
    Política:
    - UE: 15 EUR + 0.2%
    - US: 25 USD + 0.3%
    """
    if region == "UE":
        total = 15.0 + 0.002 * importe
        return f"Comisión SWIFT (UE) para {importe:.2f} EUR: {total:.2f} EUR"
    if region == "US":
        total = 25.0 + 0.003 * importe
        return f"Comisión SWIFT (US) para {importe:.2f} USD: {total:.2f} USD"
    return "Región no soportada. Usa 'UE' o 'US'."

def calcular_comision_sepa(importe_eur: float) -> str:
    """
    SEPA: gratis si importe >= 1000 EUR; si no, 2 EUR.
    """
    fee = 0.0 if importe_eur >= 1000 else 2.0
    return f"Comisión SEPA para {importe_eur:.2f} EUR: {fee:.2f} EUR"

def calcular_conversion(spread_pct: float, tipo_base: float, cantidad_base: float, pair: Optional[str]=None) -> str:
    """
    Devuelve tipo efectivo (tipo_base*(1+spread)) y cantidad convertida.
    """
    tipo_efectivo = tipo_base * (1.0 + spread_pct)
    convertido = cantidad_base * tipo_efectivo
    par = f" ({pair})" if pair else ""
    return f"Tipo efectivo{par}: {tipo_efectivo:.6f}; Resultado: {convertido:.4f}"

async def save_state(ctx: Context, response: str):
    async with ctx.store.edit_state() as ctx_state:
        ctx_state["state"]["final_response"] = response
    return "Response recorded."

swift_tool = FunctionTool.from_defaults(
    fn=calcular_comision_swift,
    name="calcular_comision_swift",
    description="Calcula comisión SWIFT para UE o US según importe."
)
sepa_tool = FunctionTool.from_defaults(
    fn=calcular_comision_sepa,
    name="calcular_comision_sepa",
    description="Calcula comisión SEPA según importe (gratis >=1000 EUR; si no, 2 EUR)."
)
fx_calc_tool = FunctionTool.from_defaults(
    fn=calcular_conversion,
    name="calcular_conversion_fx",
    description="Calcula tipo efectivo y resultado con spread, tipo base y cantidad."
)

save_state_tool = FunctionTool.from_defaults(
    fn=save_state,
    name="save_state",
    description="Guarda la respuesta final en el estado."
)

# ---------------------------------------------------------------------------
# Agentes del Workflow (todos comparten 'policies_search')
# ---------------------------------------------------------------------------
TRANSFER_SYSTEM = (
 "Eres agente especializado en transferencias (SWIFT/SEPA). "
 "PASOS: (1) Llama SIEMPRE a 'policies_search' para citar la norma; "
 "(2) Si hay importe/región, usa 'calcular_comision_swift' o 'calcular_comision_sepa'; "
 "(3) Responde TÚ con cifra final + cita breve (1 línea). "
)

transfer_agent = FunctionAgent(
    name="TransferenciasAgent",
    description="agente especializado en transferencias.",
    tools=[policy_tool, swift_tool, sepa_tool, save_state_tool],
    # can_handoff_to=[],  # <-- SIN handoff
)
transfer_agent.system_prompt = TRANSFER_SYSTEM

REFUNDS_SYSTEM = (
 "Eres agente especializado en reembolsos. "
 "Llama SIEMPRE a 'policies_search' para plazos/comisión/riesgo y responde TÚ con síntesis + cita breve. "
)

refunds_agent = FunctionAgent(
    name="ReembolsosAgent",
    description="agente especializado en reembolsos.",
    tools=[policy_tool, save_state_tool],
    # can_handoff_to=[],  # <-- SIN handoff
)
refunds_agent.system_prompt = REFUNDS_SYSTEM

CAMBIOS_SYSTEM = (
 "Eres especialista en FX. "
 "Llama SIEMPRE a 'policies_search' para obtener el spread; luego usa 'calcular_conversion_fx'. "
 "Responde TÚ con tipo efectivo + resultado + cita breve. "
)

cambios_agent = FunctionAgent(
    name="CambiosAgent",
    description="agente especializado en FX.",
    tools=[policy_tool, fx_calc_tool, save_state_tool],
    can_handoff_to=[],  # <-- SIN handoff
)
cambios_agent.system_prompt = CAMBIOS_SYSTEM

router_agent = FunctionAgent(
    name="RouterAgent",
    description=(
        "Eres un agente de IA que actúa como Orquestrador/Router. Clasifica la intención y hace handoff a un agente especializado que debe usar tools para responder la pregunta: "
        "TransferenciasAgent, ReembolsosAgent o CambiosAgent. "
        "No responde contenido de negocio al principio, solo clasifica y hace handoff.Sin embargo, la respuesta final debe ser una respuesta completa y coherente."
        "Debes mandar responder la pregunta al agente que corresponda, para que pueda responder correctamente, o delegar la tarea al siguiente agente."
    ),
    tools=[],  # sin tools
    can_handoff_to=["TransferenciasAgent", "ReembolsosAgent", "CambiosAgent"],
    
)

# ---------------------------------------------------------------------------
# Construcción del flujo
# ---------------------------------------------------------------------------
agent_flow = AgentWorkflow(
    agents=[router_agent, transfer_agent, refunds_agent, cambios_agent],
    root_agent="RouterAgent",
    initial_state={
        "final_response": ""
    }
)

print("✅ ¡Todos los agentes generales y especializados están listos!")


🧮 Herramienta de cálculo creada
✅ ¡Todas las herramientas especializadas están listas!


### ¡Probemos nuestro sistema multi-agente! 🚀

Ahora vamos a probar cómo nuestro router inteligente dirige las consultas a los agentes apropiados:

In [60]:

from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

tests = [
        # Transferencias (SWIFT UE) -> consulta + cálculo
        "Calcula la comisión total de una transferencia SWIFT de 5,000 EUR a la UE y cítame la política.",
        # SEPA (dos importes)
        "¿Cuánto cuesta una transferencia SEPA de 300 EUR y otra de 1,500 EUR?",
        # FX (EUR/USD con spread de política 0.5%)
        "Convierte 2,000 EUR a USD con tipo base 1.0870 aplicando el spread de la política para EUR/USD.",
        # Reembolsos (plazos y gestión)
        "¿Qué plazos de reembolso aplican a clientes internacionales y qué comisión de gestión corresponde?",
        # Caso encadenado (reembolso alto + riesgo)
        "Solicito reembolso de 12,500 EUR equivalentes. ¿Requiere aprobación de riesgo y cuánto tardaría?"
    ]
for q in tests:
    handler = agent_flow.run(user_msg=q)
    print("\n" + "="*80)
    print("Q:", q)
    current_agent = None
    current_tool_calls = ""
    async for event in handler.stream_events():
        if (
            hasattr(event, "current_agent_name")
            and event.current_agent_name != current_agent
        ):
            current_agent = event.current_agent_name
            print(f"\n{'='*50}")
            print(f"🤖 Agent: {current_agent}")
            print(f"{'='*50}\n")

        if isinstance(event, AgentStream):
            if event.delta:
                print(event.delta, end="", flush=True)
        elif isinstance(event, AgentInput):
            print("📥 Input:", event.input)
        elif isinstance(event, AgentOutput):
            if event.response.content:
                print("📤 Output:", event.response.content)
            if event.tool_calls:
                print(
                    "🛠️  Planning to use tools:",
                    [call.tool_name for call in event.tool_calls],
                )
        elif isinstance(event, ToolCallResult):
            print(f"🔧 Tool Result ({event.tool_name}):")
            print(f"  Arguments: {event.tool_kwargs}")
            print(f"  Output: {event.tool_output}")
        elif isinstance(event, ToolCall):
            print(f"🔨 Calling Tool: {event.tool_name}")
            print(f"  With arguments: {event.tool_kwargs}")
        
    # print("A:", str(resp))
    # state = await handler.ctx.store.get("state")
    # print("Final response:", state["final_response"])


Q: Calcula la comisión total de una transferencia SWIFT de 5,000 EUR a la UE y cítame la política.

🤖 Agent: RouterAgent

📥 Input: [ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="Current state:\n{'final_response': 'El reembolso de 12,500 EUR requiere aprobación del departamento de riesgo y puede tardar hasta 15 días laborables en procesarse.'}\n\nCurrent message:\nCalcula la comisión total de una transferencia SWIFT de 5,000 EUR a la UE y cítame la política.\n")])]
🛠️  Planning to use tools: ['handoff']
🔨 Calling Tool: handoff
  With arguments: {'to_agent': 'CambiosAgent', 'reason': 'El usuario solicita información sobre la comisión de una transferencia SWIFT y la política relacionada.'}
🔧 Tool Result (handoff):
  Arguments: {'to_agent': 'CambiosAgent', 'reason': 'El usuario solicita información sobre la comisión de una transferencia SWIFT y la política relacionada.'}
  Output: Agent CambiosAgent is now handling the reques

## 5. El Router Inteligente: El Director del Equipo 🎯

Como podemos observar en el flujo mostrado arriba, hay un agente fundamental que aún no hemos mencionado: el **router_agent**.

El **router_agent** es el encargado de analizar cada consulta del usuario y decidir, de manera inteligente, qué agente especialista debe gestionarla. Su función principal es dirigir el mensaje al agente más adecuado, optimizando así la eficiencia y precisión de las respuestas.

### 5.1 Importancia de Guardar el Estado en Sistemas Multi-Agente 💾

En sistemas multi-agente, guardar el estado permite depurar, auditar y coordinar agentes, así como recuperar el sistema tras errores. Es útil para mantener contexto, registrar decisiones y asegurar continuidad. No es necesario para consultas simples o información sensible. Por ejemplo, una herramienta como `save_state_tool` almacena respuestas parciales, cálculos y consultas externas. El estado es la memoria del sistema multi-agente.

## 6. Arquitecturas de Sistemas Multi-agentes

Ahora que entendemos los fundamentos, exploramos patrones más sofisticados.  
Cada arquitectura define **cómo se organizan y coordinan los agentes** para resolver tareas complejas.

### 1. Arquitectura Jerárquica (Supervisor → Especialistas)

El **Router/Director** recibe la consulta y decide qué agente especialista la ejecuta.  
Es clara, auditada y fácil de controlar.

<div style="text-align: center;">
  <img src="sources/router_diagram.svg" alt="router_diagram" style="width:60%; height:auto;">
</div>

---

### 2. Arquitectura Colaborativa

Los agentes se comunican **entre ellos** y comparten información, con un **coordinador** que consolida resultados.
Útil para problemas que requieren varias perspectivas.

<div style="text-align: center;">
  <img src="sources/collaborative_diagram.svg" alt=collaborative_diagram" style="width:40%; height:auto;">
</div>

---

### 3. Arquitectura Pipeline (Secuencial)

Cada agente procesa y pasa la salida al siguiente.
Muy eficiente para flujos lineales de datos o tareas repetitivas.

<div style="text-align: center;">
  <img src="sources/sequencial_diagram.svg" alt="sequencial_diagram" style="width:80%; height:auto;">
</div>
---

### 4. Arquitectura de Debate / Consenso

Dos o más agentes discuten y un **juez** selecciona o sintetiza la mejor respuesta.
Robusta frente a alucinaciones y útil en contextos críticos.

<div style="text-align: center;">
  <img src="sources/judge_diagram.svg" alt="judge_diagram" style="width:60%; height:auto;">
</div>

---

### 5. Arquitectura Blackboard (Pizarra Compartida)

Todos los agentes leen/escriben en un **espacio de memoria común**.
Ideal para problemas abiertos y creativos.

<div style="text-align: center;">
  <img src="sources/blackboard_diagram.svg" alt="Diagrama Blackboard" style="width:40%; height:auto;">
</div>

---

### 6. Arquitectura Contract Net (Mercado de Agentes)

Un **anunciante** publica la tarea; los agentes interesados hacen ofertas; se selecciona al mejor postor.
Eficiente en entornos distribuidos con recursos variables.

<div style="text-align: center;">
  <img src="sources/contract_net_diagram.svg" alt=contract_net_diagram" style="width:60%; height:auto;">
</div>

---

### Resumen rápido

| Arquitectura | Idea principal                     | Pros                   | Contras                 | Casos típicos          |
| ------------ | ---------------------------------- | ---------------------- | ----------------------- | ---------------------- |
| Jerárquica   | Supervisor asigna tareas           | Control, trazabilidad  | Cuello de botella       | QA, compliance         |
| Colaborativa | Agentes se comunican + coordinador | Diversidad de enfoques | Complejo de coordinar   | Investigación          |
| Pipeline     | Flujo secuencial                   | Simple, eficiente      | Poco flexible           | ETL, análisis de datos |
| Debate       | Varias propuestas + juez           | Robustez factual       | Coste mayor             | Contenido crítico      |
| Blackboard   | Memoria compartida                 | Fusión creativa        | Gestión de memoria      | I+D, síntesis          |
| Contract Net | Subasta de tareas                  | Asignación eficiente   | Overhead de negociación | Sistemas distribuidos  |


### 📚 Enlaces de interés

- [Multi-Agent System Architectures: A Developer's Guide (AgentHunter)](https://www.agenthunter.io/blog/multi-agent-systems-architecture)  
  Guía para entender arquitecturas jerárquicas, descentralizadas y blackboard en sistemas multi-agente.

- [Four Design Patterns for Event-Driven, Multi-Agent Systems (Confluent)](https://www.confluent.io/blog/event-driven-multi-agent-systems/)  
  Explica patrones de orquestador, colaboración y mercado aplicados a sistemas distribuidos.

- [Contract Net Protocol (Wikipedia)](https://en.wikipedia.org/wiki/Contract_Net_Protocol)  
  Descripción del protocolo clásico de subasta y asignación de tareas entre agentes.


## 9. Recursos para Expandir tu Base de Datos 📚

Para practicar con documentos reales, aquí tienes algunas fuentes públicas donde puedes descargar PDFs:

### Fuentes Recomendadas:

1. **📜 EUR-Lex / Diario Oficial de la UE**
   - Documentos legales multilingües
   - Ideal para practicar con políticas y regulaciones

2. **🏛️ Oficina de Publicaciones de la UE (op.europa.eu)**
   - Informes, guías y estudios oficiales
   - Gran variedad temática

3. **📊 SEC EDGAR (Estados Unidos)**
   - Informes financieros 10-K/10-Q
   - Perfecto para análisis financiero

4. **🔬 arXiv**
   - Papers científicos en PDF
   - Excelente para contenido técnico

5. **🏥 PubMed Central (Open Access)**
   - Artículos biomédicos
   - Ideal para dominio médico

6. **🌍 UN/World Bank/OECD**
   - Informes y análisis internacionales
   - Datos económicos y sociales


## 10. FAQ y Mejores Prácticas 🤝

**¿Por qué ChromaDB?**  
Por su instalación sencilla (`pip install chromadb`), persistencia local sin infraestructura extra, integración nativa con LlamaIndex y utilidad para prototipos y aprendizaje.  

**¿Cuántos documentos recuperar (`top-k`)?**  
Generalmente entre 3 y 5 para respuestas concretas, 5 a 8 para análisis más detallados y más de 8 solo en investigaciones extensas, teniendo en cuenta el límite de contexto el precio/token.  

**¿Cómo combinar colecciones?**  
Se recomienda crear un `QueryEngine` por cada tema, exponerlos como `QueryEngineTool` y dejar que un router de agentes seleccione la mejor opción.  

**¿Cómo evitar alucinaciones?**  
Con descripciones claras de herramientas, validaciones de entradas y salidas, límites en iteraciones y buen registro de logs.  

En cuanto a las **mejores prácticas**: diseña agentes especializados por dominio, valida siempre inputs y outputs, activa observabilidad desde el inicio, usa persistencia con ChromaDB y prueba continuamente con datasets etiquetados.  


## 🔍 **Extra: LlamaTrace – Observabilidad y Debugging Profesional**

**LlamaTrace** es la solución de observabilidad integrada en **LlamaIndex**, diseñada para monitorizar, analizar y optimizar agentes y aplicaciones de IA de manera estructurada y eficiente.

### ⚙️ Funcionalidades Principales de LlamaTrace

**1. 📝 Trazabilidad y Registro**
- Permite el seguimiento detallado de cada decisión y acción tomada por los agentes.
- Proporciona un historial completo de razonamiento y uso de herramientas.
- Genera logs exhaustivos de todas las interacciones.

**2. 📊 Métricas en Tiempo Real**
- Mide la latencia de cada componente del sistema.
- Calcula los costos asociados a cada consulta y uso de tokens.
- Ofrece indicadores de uso de recursos, rendimiento y calidad de las respuestas(LLM como jueces o tests).

**3. 🖥️ Debugging Visual y Analítico**
- Dispone de una interfaz web para explorar trazas y analizar el flujo de decisiones.
- Presenta visualizaciones como árboles de decisión y gráficos de rendimiento.


#### 🎯 Aplicaciones Típicas

- **Desarrollo y depuración:** Identificación de cuellos de botella, optimización de prompts y herramientas, y detección de errores lógicos.  
- **Monitoreo en producción:** Seguimiento de rendimiento, análisis de costos y control de calidad.  
- **Experimentación:** Comparación de modelos, pruebas A/B y ajuste de parámetros.  

#### 💡 Recomendaciones de Uso

- Mantenerlo activo en producción para un monitoreo continuo y detección temprana de incidencias.  
- Emplearlo en procesos de optimización y debugging para validar mejoras y resolver problemas.  

<div style="text-align: center;">
  <img src="https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/llama-index/arize.png" 
       alt="LlamaTrace Dashboard" style="width:100%; height:auto;">
</div>




## ¡Felicidades! 🎉 Has creado un sistema multi-agente avanzado

### 🏆 ¿Qué has logrado hoy?

En este notebook construiste un sistema completo: una **base vectorial con ChromaDB** para búsqueda semántica, un **conjunto de agentes especializados** en distintos dominios y un **router inteligente** que dirige cada consulta al agente adecuado. Además, aprendiste a orquestar múltiples agentes en consultas complejas y a usar persistencia para reutilizar datos sin reprocesamiento.  


### 🚀 ¿Por qué es esto revolucionario?

Porque se apoya en **inteligencia distribuida**, donde cada agente aporta su especialización; mejora la eficiencia mediante enrutamiento; permite escalar fácilmente añadiendo nuevos agentes y herramientas; aprovecha la persistencia para ahorrar recursos; y aumenta la precisión de las respuestas.  



### 🛣️ ¿Qué sigue?

En los próximos notebooks aprenderás:

- ✅ ✨ **Introducción a LLMs, agentes y tools**
- ✅ 🤖 **Sistemas multi-agente y ChromaDB**
- ⬜ 📄 **Parseo avanzado de documentos**
- ⬜ 🏗️ **Validación con Pydantic**
- ⬜ 🚀 **Despliegue en producción**


¡Esto es solo el comienzo de lo que puedes construir con IA Generativa! 🌟

**¡Sigue experimentando y construyendo el futuro! 🚀**
